In [ ]:
import json, os
import cv2
import torch
from multiprocessing import Pool, Manager
from aquabyte.data_access_utils import S3AccessUtils, RDSAccessUtils
from aquabyte.akpd import AKPD
from aquabyte.template_matching import find_matches_and_homography
from aquabyte.biomass_estimator import NormalizeCentered2D, NormalizedStabilityTransform, ToTensor, Network
from aquabyte.optics import euclidean_distance, pixel2world, depth_from_disp, convert_to_world_point
from aquabyte.akpd_scorer import generate_confidence_score
from keras.models import load_model
import boto3
import pandas as pd

import numpy as np
from matplotlib import pyplot as plt
import time

In [ ]:
# s3_access_utils = S3AccessUtils('/root/data')
# rds_access_utils = RDSAccessUtils(json.load(open(os.environ['DATA_WAREHOUSE_SQL_CREDENTIALS'])))

# df1 = pd.read_csv('/root/data/alok/biomass_estimation/playground/biomass.csv-61-00-from-2019-09-13-to-2019-09-23.csv')
# df1.index = pd.to_datetime(df1.captured_at)

In [ ]:
s3_access_utils = S3AccessUtils('/root/data')
rds_access_utils = RDSAccessUtils(json.load(open(os.environ['DATA_WAREHOUSE_SQL_CREDENTIALS'])))

query = """
    SELECT * FROM
    prod.crop_annotation cas
    INNER JOIN prod.annotation_state pas on pas.id=cas.annotation_state_id
    WHERE cas.service_id = (SELECT ID FROM prod.service where name='BATI')
    AND cas.annotation is not null
    AND cas.pen_id=61
    AND cas.group_id='staging-61'
    AND cas.captured_at between '2019-09-13' and '2019-09-21';
"""

df1 = rds_access_utils.extract_from_database(query)
aws_credentials = json.load(open(os.environ['AWS_CREDENTIALS']))
akpd = AKPD(aws_credentials)

to_tensor_transform = ToTensor()

# initialize data transforms so that we can run inference with biomass neural network
normalize_centered_2D_transform_biomass = NormalizeCentered2D()
normalized_stability_transform = NormalizedStabilityTransform()

# load neural network weights
biomass_network = torch.load('/root/data/alok/biomass_estimation/results/neural_network/2019-11-08T00:13:09/nn_epoch_798.pb')
akpd_scorer_network = load_model('/root/data/alok/biomass_estimation/playground/akpd_scorer_model_TF.h5') # make this better

In [ ]:
def generate_weight_score(row_id, left_crop_url, right_crop_url, left_crop_metadata, right_crop_metadata, akpd_keypoints, cm):
    
    # run AKPD scoring network
    input_sample = {
        'keypoints': akpd_keypoints,
        'cm': row.camera_metadata,
        'stereo_pair_id': row.id,
        'single_point_inference': True
    }
    akpd_score = generate_confidence_score(input_sample, akpd_scorer_network)

    # run biomass estimation
    input_sample = {
        'keypoints': akpd_keypoints,
        'cm': row.camera_metadata,
        'stereo_pair_id': row.id,
        'single_point_inference': True
    }
    nomralized_centered_2D_kps = \
        normalize_centered_2D_transform_biomass.__call__(input_sample)

    normalized_stability_kps = normalized_stability_transform.__call__(nomralized_centered_2D_kps)
    tensorized_kps = to_tensor_transform.__call__(normalized_stability_kps)
    akpd_weight_prediction = biomass_network(tensorized_kps['kp_input']).item() * 1e4
    
    
    return akpd_score, akpd_weight_prediction


In [ ]:
weight_score_dict = {}

args = []
count = 0
for idx, row in df1.iterrows():
    left_crop_url, right_crop_url = row.left_crop_url, row.right_crop_url
    left_crop_metadata, right_crop_metadata = row.left_crop_metadata, row.right_crop_metadata,
    cm = row.camera_metadata
    akpd_keypoints = row.annotation
    row_id = idx
    akpd_score, akpd_weight_prediction = generate_weight_score(row_id, left_crop_url, right_crop_url, left_crop_metadata, right_crop_metadata, akpd_keypoints, cm)
    weight_score_dict[row_id] = {
        'akpd_score': akpd_score,
        'akpd_weight_prediction': akpd_weight_prediction,
    }
    
    if count % 100 == 0:
        print(count)
    count += 1

In [ ]:
weights, akpd_scores = [], []
for idx, row in df1.iterrows():
    if idx in weight_score_dict.keys():
        weight = weight_score_dict[idx]['akpd_weight_prediction']
        weights.append(weight)
        akpd_score = weight_score_dict[idx]['akpd_score']
        akpd_scores.append(akpd_score)
    else:
        weights.append(None)
        akpd_scores.append(None)

df1['weight'] = weights
df1['akpd_score'] = akpd_scores
df1.index = pd.to_datetime(df1.captured_at)
df1['ts'] = df1.captured_at
df1 = df1.sort_values('ts')

<h1> Generate Other DF </h1>

In [ ]:
s3_access_utils = S3AccessUtils('/root/data')
rds_access_utils = RDSAccessUtils(json.load(open(os.environ['DATA_WAREHOUSE_SQL_CREDENTIALS'])))

df2 = pd.read_csv('/root/data/alok/biomass_estimation/playground/61_sample.biomass_output-CORE-404.1f.csv')
df2.index = pd.to_datetime(df2.captured_at)
df2['ts'] = df2.captured_at
df2 = df2.sort_values('ts')

In [ ]:
mask = df1.captured_at < '2019-09-21'
df1[mask & (df1.akpd_score > 0.9)].weight.mean()

In [ ]:
mask = df2.captured_at < '2019-09-21'
df2[mask & (df2.akpd_score > 0.9)].weight.mean()

In [ ]:
df1.left_crop_url.iloc[0]

In [ ]:
tdf1 = df1.copy(deep=True)
tdf2 = df2.copy(deep=True)
tdf1['adj_left_crop_url'] = tdf1.left_crop_url.apply(lambda x: x.replace('https://aquabyte-crops.s3.eu-west-1.amazonaws.com/environment=staging/', 
                                                                         'https://s3-eu-west-1.amazonaws.com/aquabyte-crops-test/environment=production/'))
common_urls = list(set(tdf1.adj_left_crop_url).intersection(set(tdf2.left_crop_url)))
tdf1 = tdf1[tdf1.adj_left_crop_url.isin(common_urls)].copy(deep=True)
tdf2 = tdf2[tdf2.left_crop_url.isin(common_urls)].copy(deep=True)


In [ ]:
mask = (tdf1.akpd_score > 0.9) & (tdf2.akpd_score > 0.9)
tdf1[mask].weight - tdf2[mask].weight

In [ ]:
plt.hist(tdf1[mask].weight - tdf2[mask].weight, bins=200)
plt.show()

In [ ]:
json.loads(tdf2[mask].annotation.iloc[0])

In [ ]:
bp = 'PELVIC_FIN'
disps1 = []
for idx, row in tdf1[mask].iterrows():
    ann = row.annotation
    left_kps = {item['keypointType']: [item['xFrame'], item['yFrame']] for item in ann['leftCrop']}
    right_kps = {item['keypointType']: [item['xFrame'], item['yFrame']] for item in ann['rightCrop']}
    disp = left_kps[bp][0] - right_kps[bp][0]
    disps1.append(disp)
    
disps2 = []
for idx, row in tdf2[mask].iterrows():
    ann = json.loads(row.annotation)
    left_kps = {item['keypointType']: [item['xFrame'], item['yFrame']] for item in ann['leftCrop']}
    right_kps = {item['keypointType']: [item['xFrame'], item['yFrame']] for item in ann['rightCrop']}
    disp = left_kps[bp][0] - right_kps[bp][0]
    disps2.append(disp)
    
print(np.mean(np.array(disps1) - np.array(disps2)))

In [ ]:
cm1 = df1.camera_metadata.iloc[0]

In [ ]:
cm2 = {
    'focalLengthPixel': 3995.5062171346935,
    'pixelCountWidth': 4096,
    'pixelCountHeight': 3000,
    'imageSensorWidth': 0.01412,
    'imageSensorHeight': 0.01035,
    'baseline': 0.10152658650444619,
    'focalLength': 3995.5062171346935 * 3.45e-6
}

In [ ]:
bp1, bp2 = 'UPPER_LIP', 'EYE'
tdf1.loc[mask, 'world_keypoints'] = tdf1[mask].annotation.apply(lambda x: pixel2world(x['leftCrop'], 
                                                  x['rightCrop'], cm1))

tdf1.loc[mask, 'length'] = tdf1[mask].world_keypoints.apply(lambda x: euclidean_distance(x[bp1], x[bp2]))

tdf2.loc[mask, 'world_keypoints'] = tdf2[mask].annotation.apply(lambda x: pixel2world(json.loads(x)['leftCrop'], 
                                                  json.loads(x)['rightCrop'], cm1))
tdf2.loc[mask, 'length'] = tdf2[mask].world_keypoints.apply(lambda x: euclidean_distance(x[bp1], x[bp2]))

In [ ]:
(tdf1[mask].length - tdf2[mask].length).median()

In [ ]:
tdf1.length.mean()

In [ ]:
plt.figure(figsize=(20, 10))
plt.hist(tdf1[mask].length - tdf2[mask].length, bins=50)
plt.show()

In [ ]:
def load_params(params_file):
    params = json.load(open(params_file))
    cameraMatrix1 = np.array(params['CameraParameters1']['IntrinsicMatrix']).transpose()
    cameraMatrix2 = np.array(params['CameraParameters2']['IntrinsicMatrix']).transpose()

    distCoeffs1 = params['CameraParameters1']['RadialDistortion'][0:2] + \
                   params['CameraParameters1']['TangentialDistortion'] + \
                   [params['CameraParameters1']['RadialDistortion'][2]]
    distCoeffs1 = np.array(distCoeffs1)

    distCoeffs2 = params['CameraParameters2']['RadialDistortion'][0:2] + \
                   params['CameraParameters2']['TangentialDistortion'] + \
                   [params['CameraParameters2']['RadialDistortion'][2]]
    distCoeffs2 = np.array(distCoeffs2)

    R = np.array(params['RotationOfCamera2']).transpose()
    T = np.array(params['TranslationOfCamera2']).transpose()

    imageSize = (4096, 3000)
    
    # perform rectification
    (R1, R2, P1, P2, Q, leftROI, rightROI) = cv2.stereoRectify(cameraMatrix1, distCoeffs1, cameraMatrix2, distCoeffs2, imageSize, R, T, None, None, None, None, None, cv2.CALIB_ZERO_DISPARITY, 0)
    

    left_maps = cv2.initUndistortRectifyMap(cameraMatrix1, distCoeffs1, R1, P1, imageSize, cv2.CV_16SC2)
    right_maps = cv2.initUndistortRectifyMap(cameraMatrix2, distCoeffs2, R2, P2, imageSize, cv2.CV_16SC2)
    
    return left_maps, right_maps, cameraMatrix1, distCoeffs1, R1, P1

In [ ]:
tdf1[mask].camera_metadata.iloc[0]

In [ ]:
matlab_params_f = '/root/data/alok/biomass_estimation/playground/2020-01-14T00_00_00Z_L40013180_R40029775_stereo-parameters.json'
circular_params_f = '/root/data/alok/biomass_estimation/playground/EstimatedStereoCamParams_L40013180_R40029775.json'

In [ ]:
left_maps_m, right_maps_m, cameraMatrix1_m, distCoeffs1_m, R1_m, P1_m = load_params(matlab_params_f)
left_maps_c, right_maps_c, cameraMatrix1_c, distCoeffs1_c, R1_c, P1_c = load_params(circular_params_f)

In [ ]:
left_maps_m[0].shape

In [ ]:
cv2.undistortPoints(np.array([[left_maps_m[0][923, 294]]]).astype(float), cameraMatrix1_m, distCoeffs1_m, R=R1_m, P=P1_m)[0][0]

In [ ]:
a, b = left_maps[0][923, 294] - right_maps[0][923, 294]

In [ ]:
tdf2.loc[mask, 'left_crop_metadata_m'] = tdf1[mask].left_crop_metadata
tdf2.loc[mask, 'right_crop_metadata_m'] = tdf1[mask].right_crop_metadata
new_anns = []
for idx, row in tdf2[mask].iterrows():
    captured_at = row.captured_at
    left_crop_metadata = row.left_crop_metadata_m
    right_crop_metadata = row.right_crop_metadata_m
    crop_x_coords = {'leftCrop': left_crop_metadata['x_coord'], 'rightCrop': right_crop_metadata['x_coord']}
    crop_y_coords = {'leftCrop': left_crop_metadata['y_coord'], 'rightCrop': right_crop_metadata['y_coord']}
    
    ann = json.loads(row.annotation)
    new_ann = {'leftCrop': [], 'rightCrop': []}
    for side in ['leftCrop', 'rightCrop']:
        for item in ann[side]:
            bp = item['keypointType']
            x, y = item['xFrame'], item['yFrame']
            maps = left_maps_c if side == 'leftCrop' else right_maps_c
            x_new, y_new = cv2.undistortPoints(np.array([[maps[0][y, x]]]).astype(float), 
                                cameraMatrix1_m, distCoeffs1_m, R=R1_m, P=P1_m)[0][0]
            new_ann[side].append({
                'keypointType': bp,
                'xFrame': x_new,
                'yFrame': y_new,
                'xCrop': x_new - crop_x_coords[side],
                'yCrop': y_new - crop_y_coords[side]
            })
    new_anns.append(new_ann)
    




In [ ]:
bp1, bp2 = 'UPPER_LIP', 'PELVIC_FIN'
tdf1.loc[mask, 'world_keypoints'] = tdf1[mask].annotation.apply(lambda x: pixel2world(x['leftCrop'], 
                                                  x['rightCrop'], cm1))

tdf1.loc[mask, 'length'] = tdf1[mask].world_keypoints.apply(lambda x: euclidean_distance(x[bp1], x[bp2]))

tdf2.loc[mask, 'world_keypoints'] = tdf2[mask].annotation.apply(lambda x: pixel2world(json.loads(x)['leftCrop'], 
                                                  json.loads(x)['rightCrop'], cm1))
tdf2.loc[mask, 'length'] = tdf2[mask].world_keypoints.apply(lambda x: euclidean_distance(x[bp1], x[bp2]))

In [ ]:
new_world_keypoints = [pixel2world(x['leftCrop'], x['rightCrop'], cm1) for x in new_anns]
tdf2.loc[mask, 'new_length'] = [euclidean_distance(x[bp1], x[bp2]) for x in new_world_keypoints]
tdf2.loc[mask, 'new_anns'] = new_anns
tdf2.loc[mask, 'new_world_keypoints'] = new_world_keypoints

In [ ]:
plt.figure(figsize=(20, 10))
plt.hist(tdf2[mask].length.values - tdf2[mask].new_length.values)
plt.show()

In [ ]:
# get circular + matlab crop urls and annotations
ts = tdf2[mask & ((tdf2.length - tdf2.new_length).abs() > 0.02)].captured_at.iloc[0]
left_crop_url_c = tdf2[mask & ((tdf2.length - tdf2.new_length).abs() > 0.02)].left_crop_url.iloc[0]
right_crop_url_c = tdf2[mask & ((tdf2.length - tdf2.new_length).abs() > 0.02)].right_crop_url.iloc[0]
ann_c = json.loads(tdf2[mask & ((tdf2.length - tdf2.new_length).abs() > 0.02)].annotation.iloc[0])

left_crop_url_m = tdf1[mask & (tdf1.captured_at == ts)].left_crop_url.iloc[0]
right_crop_url_m = tdf1[mask & (tdf1.captured_at == ts)].right_crop_url.iloc[0]
ann_m = tdf2[mask & ((tdf2.length - tdf2.new_length).abs() > 0.02)].new_anns.iloc[0]



In [ ]:
def display_crops(left_crop_url, right_crop_url, ann, overlay_keypoints=True, show_labels=False):
    fig, axes = plt.subplots(2, 1, figsize=(20, 20))
    left_image_f, bucket, left_image_key = s3_access_utils.download_from_url(left_crop_url)
    right_image_f, _, right_image_key = s3_access_utils.download_from_url(right_crop_url)
    left_image = plt.imread(left_image_f)
    right_image = plt.imread(right_image_f)
    print(right_image.shape)
    axes[0].imshow(left_image)
    axes[1].imshow(right_image)
    
    left_keypoints = {item['keypointType']: [item['xCrop'], item['yCrop']] for item in ann['leftCrop']}
    right_keypoints = {item['keypointType']: [item['xCrop'], item['yCrop']] for item in ann['rightCrop']}
    if overlay_keypoints:
        for bp, kp in left_keypoints.items():
            axes[0].scatter([kp[0]], [kp[1]], color='red', s=1)
            if show_labels:
                axes[0].annotate(bp, (kp[0], kp[1]), color='red')
        for bp, kp in right_keypoints.items():
            axes[1].scatter([kp[0]], [kp[1]], color='red', s=1)
            if show_labels:
                axes[1].annotate(bp, (kp[0], kp[1]), color='red')
    plt.show()

In [ ]:
display_crops(left_crop_url_c, right_crop_url_c, ann_c)

In [ ]:
display_crops(left_crop_url_m, right_crop_url_m, ann_m)

In [ ]:
tdf1.left_crop_metadata.iloc[0]

In [ ]:
tdf1.left_crop_url.iloc[0]

In [ ]:
xFrame, yFrame = [1794 + 2718 - 2519, 297 + 1903 - 383]
print(xFrame, yFrame)

In [ ]:
ptsTemp = np.array([], dtype='float32')
rtemp = ttemp = np.array([0,0,0], dtype='float32')
ptsOut = cv2.undistortPoints(np.array([yFrame, xFrame]).astype(float), cameraMatrix1_m, distCoeffs1_m)
ptsTemp = cv2.convertPointsToHomogeneous( ptsOut );
output = cv2.projectPoints( ptsTemp, rtemp, ttemp, cameraMatrix1, distCoeffs1_m, ptsOut );

In [ ]:
output

In [ ]:
tdf1.iloc[0]

In [ ]:
tdf1.iloc[0].base_key

In [ ]:
left_maps_m[0][yFrame, xFrame]

In [ ]:
tdf1.left_crop_metadata.iloc[0]

In [ ]:
left_maps_m[0][xFrame, yFrame]

In [ ]:
cv2.undistortPoints(np.array([[1981, 1806]]).astype(float), cameraMatrix1_m, distCoeffs1_m, R=R1_m, P=P1_m)[0][0]



In [ ]:
1743 + 238, 286+1520

In [ ]:
left_maps_m[0][1806, 1981]

In [ ]:
left_maps_m[0][1981, 1806]

In [ ]:
s3_access_utils = S3AccessUtils('/root/data')
rds_access_utils = RDSAccessUtils(json.load(open(os.environ['DATA_WAREHOUSE_SQL_CREDENTIALS'])))

query = """
    SELECT * FROM
    prod.crop_annotation cas
    INNER JOIN prod.annotation_state pas on pas.id=cas.annotation_state_id
    WHERE cas.service_id = (SELECT ID FROM prod.service where name='BATI')
    AND cas.annotation is not null
    AND cas.pen_id=88
    AND cas.captured_at between '2020-02-20' and '2020-02-21';
"""

df = rds_access_utils.extract_from_database(query)
aws_credentials = json.load(open(os.environ['AWS_CREDENTIALS']))
akpd = AKPD(aws_credentials)

to_tensor_transform = ToTensor()

# initialize data transforms so that we can run inference with biomass neural network
normalize_centered_2D_transform_biomass = NormalizeCentered2D()
normalized_stability_transform = NormalizedStabilityTransform()

# load neural network weights
biomass_network = torch.load('/root/data/alok/biomass_estimation/results/neural_network/2019-11-08T00:13:09/nn_epoch_798.pb')
akpd_scorer_network = load_model('/root/data/alok/biomass_estimation/playground/akpd_scorer_model_TF.h5') # make this better

In [ ]:
df.left_crop_url.iloc[0]

In [ ]:
df.camera_metadata.iloc[0]

In [ ]:
df.left_crop_metadata.iloc[0]

In [ ]:
matlab_params_f = '/root/data/alok/biomass_estimation/playground/2020-02-19T03_42_04.748042000Z_L40039154_R40012648_stereo-parameters.json'
left_maps_m, right_maps_m, cameraMatrix1_m, distCoeffs1_m, R1_m, P1_m = load_params(matlab_params_f)


In [ ]:
xFrame = 1217 + 1577
yFrame = 2024 + 781

In [ ]:
left_maps_m[0][yFrame, xFrame]

In [ ]:
1200 + 1604, 764 + 2036